In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv
/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


In [4]:
path = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv"


# Pandas (chunksize)

In [21]:
import pandas as pd
import time

start = time.time()

chunksize = 500000  
chunks = []

for chunk in pd.read_csv(path, chunksize=chunksize):
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

end = time.time()
print(f" تم تحميل الملف بنجاح في {end - start:.2f} ثانية")
print(f"عدد الأسطر الإجمالي: {len(df)}")


 تم تحميل الملف بنجاح في 80.76 ثانية
عدد الأسطر الإجمالي: 42448764


In [30]:
memory = df.memory_usage(deep=True).sum() / (1024 ** 2)

# Dask 

In [37]:
import dask.dataframe as dd
import time

start_dask = time.time()

df_dask = dd.read_csv(path).compute()


end_dask = time.time()
print(f" الزمن المستغرق: {end_dask - start_dask:.2f} ثانية")
print(f"عدد الأسطر الإجمالي: {len(df_dask)}")

 الزمن المستغرق: 98.63 ثانية
عدد الأسطر الإجمالي: 42448764


In [39]:
memory_dask = df_dask.memory_usage(deep=True).sum() / (1024 ** 2)

# Compressed (Gzip) 

In [15]:
!gzip /kaggle/working/2019-Oct.csv


In [18]:
start_time_gzip = time.time()
df_gzip = pd.read_csv('/kaggle/working/2019-Oct.csv.gz', compression='gzip')
end_time_gzip = time.time()
print(f" الزمن المستغرق: {end_time_gzip - start_time_gzip:.2f} ثانية")
print(f"عدد الأسطر الإجمالي: {len(df_gzip)}")

 الزمن المستغرق: 143.06 ثانية
عدد الأسطر الإجمالي: 42448764


In [24]:
memory_gzip = df_gzip.memory_usage(deep=True).sum() / (1024 ** 2)

# comparison

In [41]:
data = {
    "Method": ["Pandas (chunksize)", "Dask", "Compressed (Gzip)"],
    "Time (seconds)": [end - start, end_dask - start_dask, end_time_gzip - start_time_gzip],
    "Total Memory Used (MB)": [memory, memory_dask, memory_gzip]
}
df_comparison = pd.DataFrame(data)
print("\nThe comparison:\n")
print(df_comparison.to_string(index=False))


The comparison:

            Method  Time (seconds)  Total Memory Used (MB)
Pandas (chunksize)       80.762608            15743.616797
              Dask       98.630115             6627.627619
 Compressed (Gzip)      143.058658            15743.616797
